# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port blair,11.6667,92.7500,31.02,74,75,7.72,IN,1690613884
1,1,east london,-33.0153,27.9116,17.52,82,20,7.72,ZA,1690613884
2,2,edinburgh of the seven seas,-37.0676,-12.3116,14.58,84,100,12.95,SH,1690613885
3,3,seven pagodas,12.6264,80.1722,34.03,45,100,7.57,IN,1690613885
4,4,gerash,27.6652,54.1371,36.08,28,6,1.81,IR,1690613886


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )
# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values

# Ideal city weather:
# Temperatures between 21°C and 29°C
# Humidity less than 80%
# Cloudiness less than 35%
# Wind Speed less than 7 m/s

city_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 29)
                                       & (city_data_df["Max Temp"] > 21)
                                       & (city_data_df["Humidity"] < 80)
                                       & (city_data_df["Cloudiness"] < 35)
                                       & (city_data_df["Wind Speed"] < 7)
                                      ]

# Drop any rows with null values
city_weather_df.dropna()


# Display sample data
city_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,mitsamiouli,-11.3847,43.2844,26.16,65,20,2.06,KM,1690613887
29,29,port mathurin,-19.6833,63.4167,23.43,78,7,6.04,MU,1690613897
34,34,yeppoon,-23.1333,150.7333,21.88,76,15,3.81,AU,1690613900
36,36,whitehorse,60.7161,-135.0538,21.43,48,20,3.09,CA,1690613900
37,37,badger,64.8000,-147.5333,24.95,66,0,3.60,US,1690613901
...,...,...,...,...,...,...,...,...,...,...
531,531,emerald,-23.5333,148.1667,24.08,39,6,3.03,AU,1690614164
545,545,borogontsy,62.6683,131.1683,25.64,29,5,2.14,RU,1690614171
564,564,labytnangi,66.6572,66.4183,22.53,45,0,4.47,RU,1690614179
580,580,pozoblanco,38.3791,-4.8483,22.62,38,0,1.19,ES,1690614187


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,mitsamiouli,KM,-11.3847,43.2844,65,
29,port mathurin,MU,-19.6833,63.4167,78,
34,yeppoon,AU,-23.1333,150.7333,76,
36,whitehorse,CA,60.7161,-135.0538,48,
37,badger,US,64.8000,-147.5333,66,
...,...,...,...,...,...,...
531,emerald,AU,-23.5333,148.1667,39,
545,borogontsy,RU,62.6683,131.1683,29,
564,labytnangi,RU,66.6572,66.4183,45,
580,pozoblanco,ES,38.3791,-4.8483,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
            "categories": "accommodation.hotel",
            "apiKey":geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mitsamiouli - nearest hotel: Hotel le Paradis
port mathurin - nearest hotel: Escale Vacances
yeppoon - nearest hotel: Blossom's on Seaspray
whitehorse - nearest hotel: Town & Mountain Hotel
badger - nearest hotel: Hotel North Pole
diffa - nearest hotel: UNHCR guest house
mwinilunga - nearest hotel: No hotel found
antonio enes - nearest hotel: Hotel Quirimbas
mwene-ditu - nearest hotel: Hôtel KAMAS
chipinge - nearest hotel: No hotel found
tommot - nearest hotel: Томмот
sidney - nearest hotel: The Westin
karratha - nearest hotel: Karratha International Hotel
montepuez - nearest hotel: Complexo Messalo
newman - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
abeche - nearest hotel: No hotel found
kurtamysh - nearest hotel: Загородный Дом Ласточкино Гнездо
chios - nearest hotel: Aegean Sea
udachny - nearest hotel: Вилюй
nsanje - nearest hotel: No hotel found
uvinza - nearest hotel: No hotel found
majene - nearest hotel: No hotel found
san lawre

,City,Country,Lat,Lng,Humidity,Hotel Name
7,mitsamiouli,KM,-11.3847,43.2844,65,Hotel le Paradis
29,port mathurin,MU,-19.6833,63.4167,78,Escale Vacances
34,yeppoon,AU,-23.1333,150.7333,76,Blossom's on Seaspray
36,whitehorse,CA,60.7161,-135.0538,48,Town & Mountain Hotel
37,badger,US,64.8000,-147.5333,66,Hotel North Pole
...,...,...,...,...,...,...
531,emerald,AU,-23.5333,148.1667,39,Emerald Central Hotel
545,borogontsy,RU,62.6683,131.1683,29,No hotel found
564,labytnangi,RU,66.6572,66.4183,45,Семь лиственниц
580,pozoblanco,ES,38.3791,-4.8483,38,Casa Rural Abuela Teo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)